In [1]:

import ee
import geemap
import os
import geopandas as gpd


ambiente gee per windows

In [3]:
# load geojson
area = 'mnt/d/Dropbox/projects/HOLISTIC/GIS/area.geojson'

In [3]:
area_geojson = 'D:/Dropbox/projects/HOLISTIC/GIS/area.geojson'

# Caricare il file GeoJSON come GeoDataFrame
gdf = gpd.read_file(area_geojson)

# Ottenere il centroide dell'area per centrare la mappa
centroid = gdf.geometry.centroid[0]
center_lat = centroid.y
center_lon = centroid.x

# Creare una mappa centrata sull'area
m = geemap.Map(center=(center_lat, center_lon), zoom=12)

# Aggiungere il layer GeoJSON alla mappa
m.add_gdf(gdf, layer_name='Area')

# Mostrare la mappa
m


Map(center=[-74.70326637811245, 163.49524695258341], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
start_date = '2022-12-10'
end_date = '2022-12-26'



# extract the days from the start date
start_date = ee.Date(start_date)

# add 1 week to the start date
#end_date = start_date.advance(8, 'week')
end_date = ee.Date(end_date)


In [9]:
# Ottenere la bounding box del GeoDataFrame
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]

# Definire il ROI (Regione di Interesse) dalla bounding box
roi = ee.Geometry.BBox(bounds[0], bounds[1], bounds[2], bounds[3])


In [18]:

# get the image collection
# select collection s2 level2A
s2col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterDate(start_date, end_date) 
    

# print the number of images in the collection
print('Number of images in the collection: ', s2col.size().getInfo())


Number of images in the collection:  5


In [25]:
from datetime import datetime


In [27]:
# Funzione per estrarre la data e stamparla in formato leggibile
def print_dates(image):
    # Ottieni la data dell'immagine
    date_info = ee.Date(image.get('system:time_start')).getInfo()
    timestamp = date_info['value']  # Ottieni il timestamp corretto
    # Converte in un formato leggibile
    readable_date = datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
    print(readable_date)
    
# Applica la funzione a tutte le immagini della collezione
s2col.aggregate_array('system:time_start').getInfo()

# Stampa le date in formato leggibile
image_list = s2col.toList(s2col.size())
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    print_dates(img)

2022-12-18 21:27:14
2022-12-18 21:27:03
2022-12-18 21:27:08
2022-12-18 21:26:56
2022-12-24 21:46:59


In [31]:
# Funzione per visualizzare ciascuna immagine nella collezione
def add_images_to_map(image):
    # Ottiene la data dell'immagine per usarla nel nome del layer
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    # Aggiungi l'immagine alla mappa con una visualizzazione standard di Sentinel-2
    visualization = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}
    m.addLayer(image.clip(roi), visualization, f'Sentinel-2 {date}')

# Converte la collezione in una lista e aggiunge ciascuna immagine alla mappa
image_list = s2col.toList(s2col.size())
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    add_images_to_map(img)

# Mostra la mappa
m

Map(bottom=429466.0, center=[-74.61016555284752, 163.24756622314456], controls=(WidgetControl(options=['positi…

In [34]:
# take the first image of the collection
first = s2col.first()
first = first.clip(roi)


m.addLayer(first, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'first')
m

Map(bottom=54209.0, center=[-74.98929798403441, 163.27606201171878], controls=(WidgetControl(options=['positio…

In [40]:
# Prendi la seconda immagine della collezione
second = ee.Image(image_list.get(3))  # Gli indici partono da 0, quindi 1 per la seconda immagine
second = second.clip(roi)

# Crea la mappa
m = geemap.Map(center=[(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2], zoom=12)

# Aggiungi la seconda immagine alla mappa (Sentinel-2 RGB)
m.addLayer(second, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 10000}, 'Second Sentinel-2 Image')
m.addLayer(roi, {'color': 'FF0000'}, 'ROI')

# Mostra la mappa
m

Map(center=[-74.70279681889666, 163.48757479546], controls=(WidgetControl(options=['position', 'transparent_bg…

In [41]:
s1col = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

print('Number of images in the collection: ', s1col.size().getInfo())

Number of images in the collection:  4


In [55]:
# print the dates
image_list = s1col.toList(s1col.size())
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    print_dates(img)
    
# Applica la funzione a tutte le immagini della collezione
s1col.aggregate_array('system:time_start').getInfo()

# Stampa le date in formato leggibile
image_list = s1col.toList(s1col.size())
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    print_dates(img)

2022-12-12 15:38:07
2022-12-14 10:30:29
2022-12-15 16:02:20
2022-12-24 15:38:06
2022-12-12 15:38:07
2022-12-14 10:30:29
2022-12-15 16:02:20
2022-12-24 15:38:06


In [56]:
# vedo la terza
# Prendi la seconda immagine della collezione
s1sec = ee.Image(image_list.get(4))  # Gli indici partono da 0, quindi 1 per la seconda immagine
s1sec = s1sec.clip(roi)

print(s1sec)


ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    162.60364609374085,
                    -74.40460164467032
                  ],
                  [
                    162.60364609374085,
                    -75.000991993123
                  ],
                  [
                    164.37150349717916,
                    -75.000991993123
                  ],
                  [
                    164.37150349717916,
                    -74.40460164467032
                  ]
                ]
              ]
            },
            "geodesic": {
              "constantValue": false
            }
          }
        }
      },
      "input": {
        "functionInvocationValue": {

In [57]:
band_names = s1sec.bandNames().getInfo()
print("Bande disponibili:", band_names)

EEException: List.get: List index must be between 0 and 3, or -4 and -1. Found 4.

In [58]:
s1sec = ee.Image(image_list.get(0))  # Gli indici partono da 0, quindi 1 per la seconda immagine
s1sec = s1sec.clip(roi)

# Visualizzazione dell'immagine SAR: solo la banda "HH"
sar_viz_params_hh = {
    'min': -30,
    'max': 0,
    'bands': ['HH'],
    'gamma': 1.5
}

# Aggiungi la banda "HH" alla mappa
m.addLayer(s1sec, sar_viz_params_hh, 'Third Sentinel-1 Image (HH)')

# Mostra la mappa
m

Map(bottom=108064.0, center=[-74.95083630838428, 163.07830810546875], controls=(WidgetControl(options=['positi…

In [59]:
# print date od s1sec
print_dates(s1sec)

2022-12-12 15:38:07


In [60]:
# define out_dir
out_dir = os.path.expanduser('D:\\Dropbox\R_projects\HOLISTIC\\out')

out_dir

# define filename with the out_dir
filename = os.path.join(out_dir, 'S1_20221212_10m.tif')
geemap.download_ee_image(s1sec, filename= filename, scale=10, region = roi, crs='EPSG:4326')


Consider adjusting `region`, `scale` and/or `dtype` to reduce the S1_20221212_10m.tif download size (raw: 2.09 GB).


S1_20221212_10m.tif: |          | 0.00/2.09G (raw) [  0.0%] in 00:00 (eta:     ?)